In [1]:
import pickle
import pickle as pkl
import json
import sqlite3 as sql
from pathlib import Path
import pandas as pd
from rdkit.Chem import Draw
from rdkit.Chem.rdMolDescriptors import CalcMolFormula, CalcExactMolWt
from rdkit.Chem.rdmolfiles import ForwardSDMolSupplier
from rdkit import Chem
from PIL import Image
import re
import copy
import gzip
from util_functions import *


In [2]:
DATA_DIR = Path('..', 'data').resolve()
INFO_PATH = DATA_DIR / 'library_static' / 'library_constituents_dataframe.pkl'
MAPPING_PATH = DATA_DIR / 'buildingblocks' / 'compound_mapping.txt'
EXP_PLAN_PATH = DATA_DIR / 'library_info' / 'synthesis_plan.json'
DB_PATH = DATA_DIR / 'db' / '50k_project.db'
STATIC_DIR = DATA_DIR / 'db' / 'static'

In [3]:
con = sql.connect(DB_PATH)
cur = con.cursor()

In [4]:
with open(MAPPING_PATH, 'r') as file:
    compound_long_names = []
    for line in file.readlines():
        compound_long_names.append(line.split()[1])

In [5]:
# lets grab our building blocks from their db (to determine protecting groups)
initiator_ids = [i[0] for i in cur.execute('SELECT id FROM buildingblocks WHERE category = ?;', 'I').fetchall()]
monomer_ids = [i[0] for i in cur.execute('SELECT id FROM buildingblocks WHERE category = ?;', 'M').fetchall()]
terminator_ids = [i[0] for i in cur.execute('SELECT id FROM buildingblocks WHERE category = ?;', 'T').fetchall()]
print(len(initiator_ids), len(monomer_ids), len(terminator_ids))

78 74 41


In [6]:
# let's try adding product A for one combination
# initiator_short, initiator_long, boc_i, cbz_i, tbu_i, tms_i\
def query_building_block(id):
    """For a building block id, retrieve short, long, #boc, #cbz, #tbu, #tms in that order from DB"""
    assert type(id) is str or type(id) is int
    result = cur.execute('SELECT short, long, boc, cbz, tbu, tms FROM buildingblocks WHERE id = ?;', [id]).fetchone()
    if result is None:
        raise ValueError(f'No entry in database for id {id}')
    return result

def count_protecting_groups(initiator, monomer, terminator, product_type):
    # we will assume the indices as 2: boc, 3: cbz, 4: tbu, 5:tms
    if product_type in ['A', 'B', 'C']:
        boc = initiator[2] + monomer[2] + terminator[2]
        cbz = initiator[3] + monomer[3] + terminator[3]
        tbu = initiator[4] + monomer[4] + terminator[4]
        tms = initiator[5] + monomer[5] + terminator[5]
    elif product_type in ['D']:
        boc = initiator[2] + terminator[2]
        cbz = initiator[3] + terminator[3]
        tbu = initiator[4] + terminator[4]
        tms = initiator[5] + terminator[5]
    elif product_type in 'E':
        boc = 2 * terminator[2]
        cbz = 2 * terminator[3]
        tbu = 2 * terminator[4]
        tms = 2 * terminator[5]
    elif product_type in ['F', 'G']:
        boc = initiator[2] + monomer[2]
        cbz = initiator[3] + monomer[3]
        tbu = initiator[4] + monomer[4]
        tms = initiator[5] + monomer[5]
    elif product_type == 'H':
        boc = monomer[2] + terminator[2]
        cbz = monomer[3] + terminator[3]
        tbu = monomer[4] + terminator[4]
        tms = monomer[5] + terminator[5]
    else:
        raise ValueError(f'Invalid product type {product_type}')

    return boc, cbz, tbu, tms

for product_type in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']:
    for i in initiator_ids:
        for m in monomer_ids:
            for t in terminator_ids:
                initiator = query_building_block(i)
                monomer = query_building_block(m)
                terminator = query_building_block(t)
                boc, cbz, tbu, tms = count_protecting_groups(initiator, monomer, terminator, product_type)
                # Let's add those entries to DB
                cur.execute('INSERT INTO virtuallibrary (initiator, monomer, terminator, type, boc, cbz, tbu, tms, initiator_long, monomer_long, terminator_long) VALUES (?,?,?,?,?,?,?,?,?,?,?)', (initiator[0], monomer[0], terminator[0], product_type, boc, cbz, tbu, tms, initiator[1], monomer[1], terminator[1]))
            # commit after going through all terminators
            con.commit()

In [ ]:
"""
still TODO
- get product mol -> image and smiles and molecular formula_1
- from pg numbers and pg dict, get the molecular formulae of deprotection products
"""

In [7]:
# open the sdf.gz
product_dicts = {}
for product_type in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']:
    counter = 0
    product_dict = {}
    with gzip.open(DATA_DIR / 'library_static' / f'product_{product_type}.sdf.gz') as file:
        supplier = ForwardSDMolSupplier(file)
        for mol in supplier:
            if mol is not None:

                smiles = Chem.MolToSmiles(mol)
                molecular_formula_1 = CalcMolFormula(mol)
                mass_1 = CalcExactMolWt(mol)
                name = mol.GetProp('_Name')
                product_dict[name] = (smiles, molecular_formula_1, mass_1)

                # # now we need to match the mol with a database entry
                # result = cur.execute('SELECT id FROM virtuallibrary WHERE long_name = ? AND type = ?;', [name, product_type]).fetchall()
                # # check if we got exactly one id back
                # try:
                #     if len(result) == 0:
                #         raise ValueError(f'No entry in DB for long name {name}')
                #     elif len(result) > 1:
                #         raise ValueError(f'Duplicate entry for long name {name} and type {product_type}. Offending entries: {result}')
                #     # sine we now have the correct id, we can add the new info obtained from the sdf file
                #     cur.execute('UPDATE virtuallibrary SET SMILES = ?, molecular_formula_1 = ?, lcms_mass_1 = ?  WHERE id = ?', (smiles, molecular_formula_1, mass_1, result[0][0]))
                # except ValueError:
                #     # only raise the error if the building block was used in 50k project
                #     if name.split(' + ')[0] in compound_long_names and name.split(' + ')[1] in compound_long_names and name.split(' + ')[2] in compound_long_names:
                #         raise ValueError(f'Failed on product {name}')
    product_dicts[product_type] = product_dict
    print(f'{product_type} done')


H done


In [26]:
for id, long, ptype in cur.execute('SELECT id, long_name, type from virtuallibrary;').fetchall():
    try:
        props = product_dicts[ptype][long]
        if props[1] != 'H2O':  # don't write anything if the water placeholder is encountered
            cur.execute('UPDATE virtuallibrary SET SMILES = ?, molecular_formula_1 = ?, lcms_mass_1 = ? WHERE id = ?;', (props[0], props[1], props[2], id))
    except KeyError:  # raised if the long name is not found in product_dict
        if ptype == 'H':  # if this was in H, it is probably ok since we cannot enumerate that with chemical sense for everything
            print(f'A KeyError occured for {long}. Please check')
        else:
            raise KeyError

In [27]:
con.commit()

In [14]:
"""A dictionary to look up protecting group properties"""
pg_dict = {
    'boc': ('C5H8O2', 100.0524),
    'cbz': ('C8H6O2', 134.0368),
    'tbu': ('C4H8', 56.0626),
    'tms': ('C3H8Si', 72.0395),
}

In [28]:
"""Finally we need to do our magic with the protecting groups"""
for id, molecular_formula_1, lcms_mass_1, boc, cbz, tbu, tms in cur.execute('SELECT id, molecular_formula_1, lcms_mass_1, boc, cbz, tbu, tms from virtuallibrary').fetchall():
    if molecular_formula_1 is not None and lcms_mass_1 is not None and molecular_formula_1 != 'H2O':  # don't evaluate for missing products (e.g. in H)
        additional_formulae = []
        additional_masses = []
        # generate all the combinations in that we need to leave PGs out
        combinations = [(i, j, k, l) for i in range(boc+1) for j in range(cbz+1) for k in range(tbu+1) for l in range(tms + 1) if sum([i,j,k,l]) > 0]
        pgname = ['boc', 'cbz', 'tbu', 'tms']
        for i in combinations:
            new_mass = lcms_mass_1 - i[0] * pg_dict['boc'][1] - i[1] * pg_dict['cbz'][1] - i[2] * pg_dict['tbu'][1] - i[3] * pg_dict['tms'][1]
            substrahend_formulae = [j * pg_dict[pgname[idx]][0] for idx, j in enumerate(i)]
            new_form = molecular_formula_1
            for s in substrahend_formulae:
                if s != '':
                    new_form = string_formula_substraction(new_form, s)
            additional_masses.append(new_mass)
            additional_formulae.append(new_form)
        additional_masses = [f'{i:.4f}' for i in additional_masses]
        if len(additional_formulae) > 0:
            cur.execute('UPDATE virtuallibrary SET molecular_formula_alt = ?, lcms_mass_alt = ? WHERE id = ?', (','.join(additional_formulae), ','.join(additional_masses), id))
con.commit()

In [ ]:
pgname = ['boc', 'cbz', 'tbu', 'tms']
tbu =2
lcms_mass_1 = 700.0
mol_form = 'C10H27O5N5Si'
combinations = [(i, j, k, l) for i in range(boc+1) for j in range(cbz+1) for k in range(tbu+1) for l in range(tms + 1) if sum([i,j,k,l]) > 0]
for i in combinations:
    new_mass = lcms_mass_1 - i[0] * pg_dict['boc'][1] - i[1] * pg_dict['cbz'][1] - i[2] * pg_dict['tbu'][1] - i[3] * pg_dict['tms'][1]
    substrahend_formulae = [j * pg_dict[pgname[idx]][0] for idx, j in enumerate(i)]
    new_form = mol_form
    for s in substrahend_formulae:
        if s != '':
            new_form = string_formula_substraction(new_form, s)

In [20]:
con.rollback()


In [ ]:
con.commit()

In [ ]:
"""A dictionary to look up protecting group properties"""
pg_dict = {
    'boc': ('C5H8O2', 100.0524),
    'cbz': ('C8H6O2', 134.0368),
    'tbu': ('C4H8', 56.0626),
    'tms': ('C3H8Si', 72.0395),
}

In [17]:
def parse_formula(formula : str) -> dict: # Formula Parsing by Aditya Matam
    def multiply(formula: dict, mul: int) -> None:
        for key in formula: formula[key] *= mul

    formDict = {}
    # PARENS
    for match in re.finditer(r"\((.*?)\)(\d*)", formula):
        parens = parse_formula(match.group(1))
        mul = match.group(2)
        if not mul: mul = 1
        multiply(parens, int(mul))
        formDict.update(parens)
    # REST
    for match in re.finditer(r"(\(?)([A-Z][a-z]?)(\d*)(\)?)", formula):
        left, elem, mul, right = match.groups()
        if left or right: continue
        if not mul: mul = 1
        if elem in formDict:
            formDict[elem] += int(mul)
        else:
            formDict[elem] = int(mul)

    return formDict

def formula_to_string(formDict):
    s = ''
    for key, value in formDict.items():
        if value == 1:
            s += key
        elif value > 1:
            s += f'{key}{value}'
    return s

def substract_formulae(minuend, substrahend):
    result = copy.deepcopy(minuend)  # we make a deepcopy to not alter the minuend
    for key, value in substrahend.items():
        result[key] -= value
    return result

def string_formula_substraction(minuend, substrahend):
    return formula_to_string(substract_formulae(parse_formula(minuend), parse_formula(substrahend)))

In [18]:
"""For existing BUILDINGBLOCK entries, use SMILES and number of protecting groups to calculate probable formulae in LCMS"""
for id, smiles, boc, cbz, tbu, tms in cur.execute('SELECT id, SMILES, boc, cbz, tbu, tms FROM buildingblocks;').fetchall():
    mol = Chem.MolFromSmiles(smiles)
    lcms_formula_1 = CalcMolFormula(mol)
    lcms_mass_1 = CalcExactMolWt(mol)
    cur.execute('UPDATE buildingblocks SET lcms_formula_1 = ?, lcms_mass_1 = ? WHERE id = ?', (lcms_formula_1, lcms_mass_1, id))
    con.commit()
    additional_formulae = []
    additional_masses = []
    for pg, pgname in zip([boc, cbz, tbu, tms], ['boc', 'cbz', 'tbu', 'tms']):
        for i in range(pg):
            # if boc == 0, this will not evaluate
            additional_formulae.append(string_formula_substraction(lcms_formula_1, pg_dict[pgname][0]))
            additional_masses.append(lcms_mass_1 - pg_dict[pgname][1])
    additional_masses = [f'{i:.4f}' for i in additional_masses]
    if len(additional_formulae) > 0:
        cur.execute('UPDATE buildingblocks SET lcms_formula_alt = ?, lcms_mass_alt = ? WHERE id = ?', (','.join(additional_formulae), ','.join(additional_masses), id))
        con.commit()

In [ ]:
ex1 = 'C10H5O'
ex2 = 'C3H'
formula_to_string(substract_formulae(parse_formula(ex1), parse_formula(ex2)))

In [ ]:
con.commit()

In [ ]:
con.close()